In [30]:
import tensorflow as tf
import numpy as np

# set Ture during training, False during testing
TRAINING = True


def identity_block(X_input, kernel_size, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    with tf.name_scope("id_block_stage"+str(stage)):
        filter1, filter2, filter3 = filters
        X_shortcut = X_input

        # First component of main path
        x = tf.layers.conv2d(X_input, filter1,
                 kernel_size=(1, 1), strides=(1, 1),name=conv_name_base+'2a')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base+'2a', training=TRAINING)
        x = tf.nn.relu(x)

        # Second component of main path
        x = tf.layers.conv2d(x, filter2, (kernel_size, kernel_size),
                                 padding='same', name=conv_name_base+'2b')
        # batch_norm2 = tf.layers.batch_normalization(conv2, axis=3, name=bn_name_base+'2b', training=TRAINING)
        x = tf.nn.relu(x)

        # Third component of main path
        x = tf.layers.conv2d(x, filter3, kernel_size=(1, 1),name=conv_name_base+'2c')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2c', training=TRAINING)

        # Final step: Add shortcut value to main path, and pass it through a RELU activation
        X_add_shortcut = tf.add(x, X_shortcut)
        add_result = tf.nn.relu(X_add_shortcut)

    return add_result


def convolutional_block(X_input, kernel_size, filters, stage, block, stride = 2):
    #change the shape of output so that it can do sum process with shotcut
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    with tf.name_scope("conv_block_stage" + str(stage)):

        # Retrieve Filters
        filter1, filter2, filter3 = filters

        # Save the input value
        X_shortcut = X_input

        # First component of main path
        x = tf.layers.conv2d(X_input, filter1,
                                 kernel_size=(1, 1),
                                 strides=(stride, stride),
                                 name=conv_name_base+'2a')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base+'2a', training=TRAINING)
        x = tf.nn.relu(x)

        # Second component of main path
        x = tf.layers.conv2d(x, filter2, (kernel_size, kernel_size), name=conv_name_base + '2b',padding='same')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2b', training=TRAINING)
        x = tf.nn.relu(x)

        # Third component of main path
        x = tf.layers.conv2d(x, filter3, (1, 1), name=conv_name_base + '2c')
        x = tf.layers.batch_normalization(x, axis=3, name=bn_name_base + '2c', training=TRAINING)

        # SHORTCUT PATH
        X_shortcut = tf.layers.conv2d(X_shortcut, filter3, (1,1),
                                      strides=(stride, stride), name=conv_name_base + '1')
        X_shortcut = tf.layers.batch_normalization(X_shortcut, axis=3, name=bn_name_base + '1', training=TRAINING)

        # Final step: Add shortcut value to main path, and pass it through a RELU activation
        X_add_shortcut = tf.add(X_shortcut, x)
        add_result = tf.nn.relu(X_add_shortcut)

    return add_result



def ResNet50_reference(X, classes= 2):
#     x = tf.pad(X, tf.constant([[0, 0],[3, 3,], [3, 3], [0, 0]]), "CONSTANT")

#     assert(x.shape == (x.shape[0], 70, 70, 1))

    # stage 1
    x = tf.layers.conv2d(X, filters=64, kernel_size=(7, 7), strides=(2, 2), name='conv1')
    x = tf.layers.batch_normalization(x, axis=3, name='bn_conv1')
    x = tf.nn.relu(x)
    # local response normalization
    x = tf.nn.lrn(x, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    x = tf.layers.max_pooling2d(x, pool_size=(3, 3),strides=(2, 2))
    


    # stage 2
    x = convolutional_block(x, kernel_size=3, filters=[64, 64, 256], stage=2, block='a', stride=1)
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    # stage 3
    x = convolutional_block(x, kernel_size=3, filters=[128,128,512],
                                            stage=3, block='a', stride=2)
    x = identity_block(x, 3, [128,128,512], stage=3, block='b')
    x = identity_block(x, 3, [128,128,512], stage=3, block='c')
    x = identity_block(x, 3, [128,128,512], stage=3, block='d')

    # stage 4
    x = convolutional_block(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='a', stride=2)
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    # stage 5
    x = convolutional_block(x,kernel_size=3,filters=[512, 512, 2048], stage=5, block='a', stride=2)
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    #x = tf.layers.average_pooling2d(x, pool_size=(2, 2), strides=(1,1))
    # local response normalization
    x = tf.nn.lrn(x, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    x = tf.layers.max_pooling2d(x, pool_size=(2, 2), strides=(1,1))

    flatten = tf.layers.flatten(x, name='flatten')
    # set 1 when testing
    keep_prob = 1
    # dropout
    drop = tf.nn.dropout(flatten, keep_prob)
    dense1 = tf.layers.dense(drop, units=50, activation=tf.nn.relu)
    logits = tf.layers.dense(dense1, units=2, activation=tf.nn.softmax)
    return logits


In [31]:
def losses(logits, labels, name):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        if name=='train':
            tf.summary.scalar(scope.name + '/train_loss', loss)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_loss', loss)
    return loss
 
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
def evaluation(logits, labels, name):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_accuracy', accuracy)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_accuracy', accuracy)
    return accuracy

# def change_recall(logits,value):
#     def re(elem):
#         if elem[tf.argmax(elem)]>=value:
#             return 1
#         else:
#             return 0
#     predict = tf.map_fn(re, logits)
#     return predict

def recall_precision(logits, labels, name):
#     logits = tf.cast(logits, tf.int64)
    labels = tf.cast(labels, tf.int64)
    predict = tf.cast(logits,tf.int64)
#     predict = tf.arg_max(logits,1)
#     predict = tf.cast(change_recall(logits,0.79), tf.int64)
    with tf.variable_scope('recall_precision') as scope:
        TP = tf.count_nonzero(predict * labels)
        TN = tf.count_nonzero((predict - 1) * (labels - 1))
        FN = tf.count_nonzero(predict * (labels - 1))
        FP = tf.count_nonzero((predict - 1) * labels)
        precision = tf.divide(TP, TP + FP)
        recall = tf.divide(TP, TP + FN)
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)
        #f1 = tf.cast(f1, dtype=tf.float32)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_precision', precision)
            tf.summary.scalar(scope.name + '/train_recall', recall)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_precision', precision)
            tf.summary.scalar(scope.name + '/valid_recall', recall)
    return precision, recall

In [32]:
import os
 
def get_files(path_pos,path_neg,label_pos,label_neg):
    TC = []
    label_TC = []
    nonTC = []
    label_nonTC = []
    # data loader
    file_dir_TC=path_pos
    file_dir_nonTC=path_neg
    #len_valTC=len(os.listdir(file_dir_TC))
    for file in os.listdir(file_dir_TC):
        name = file.split('_')
        if name[0] == label_pos:
            TC.append(file_dir_TC + file)
            label_TC.append(1)
    for file in os.listdir(file_dir_nonTC):
        name = file.split('_')
        if name[0] == label_neg:
            nonTC.append(file_dir_nonTC + file)
            label_nonTC.append(0)
    print("There are %d TC\nThere are %d nonTC" % (len(TC), len(nonTC)))
 
    # shuffle
    image_list = np.hstack((TC, nonTC))
    label_list = np.hstack((label_TC, label_nonTC))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()    
    np.random.shuffle(temp)
 
    image_list = list(temp[:, 0])
    label_list = list(temp[:, 1])
    label_list = [int(i) for i in label_list]
 
    return image_list, label_list
 
# img_list,label_list = get_files(file_dir)
 
# batch
def get_batch(image, label, image_W, image_H, batch_size, capacity):   
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    image_contents = tf.read_file(input_queue[0])
    label = input_queue[1]
    image = tf.image.decode_jpeg(image_contents, channels=1)
 
    # resize
    image = tf.image.resize_images(image, [image_H, image_W], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    # image = tf.image.per_image_standardization(image)  
    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,  
                                              capacity=capacity)
  
    return image_batch, label_batch

In [34]:
##validation
import os
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
tf.device("/device:GPU:0")
N_CLASSES = 2 
IMG_W = 64  # resize
IMG_H = 64
BATCH_SIZE = 1024
CAPACITY = 20000
MAX_STEP = 1000

def change_recall(val_logits):
    predict=[]
    for i in val_logits:
#         if i[1]>=0.79:
#             predict.append(1)
#         else:
#             predict.append(0)
            predict.append(1)
        
    predict = np.float32(np.reshape(predict,1024))
    return predict

file_dir_valTC='/home/ubuntu/data/valTC_cp/'
file_dir_valnonTC='/home/ubuntu/data/valnonTC/'

valid, valid_label = get_files(file_dir_valTC,file_dir_valnonTC,'valTC','valnonTC')
logs_valid_dir='./valid_'

valid_batch,valid_label_batch=get_batch(valid,
                                valid_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)

#with tf.Graph().as_default():
ckpt_path = './train_resnet_lrnaug_balanced/model.ckpt-1564000'

# x = tf.placeholder(tf.float32, [BATCH_SIZE,IMG_W,IMG_H,1])
# y = tf.placeholder(tf.float32, [BATCH_SIZE])
# y = tf.cast(y,tf.int64)
pred = tf.placeholder(tf.float32, [1024])
train_logits = ResNet50_reference(valid_batch)
valid_acc = evaluation(train_logits, valid_label_batch, 'valid')
val_recall, val_precision = recall_precision(pred, valid_label_batch, 'valid')
#val_recall, val_precision = recall_precision(pred, valid_label_batch, 'valid')
saver = tf.train.Saver()
    
with tf.Session() as sess:
    #valid_batch,valid_label_batch=sess.run([valid_batch,valid_label_batch])
    sess.run(tf.global_variables_initializer())
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#     variables = tf.contrib.framework.get_variables_to_restore()
#     variables_to_resotre = [v for v in varialbes if v.name.split('/')[0]!='softmax_linear']

    saver.restore(sess,ckpt_path)
    print('--------restore done---------')
    summary_op = tf.summary.merge_all() 

    valid_writer = tf.summary.FileWriter(logs_valid_dir, sess.graph)
  
    print('---------validation start----------')
    for step in np.arange(MAX_STEP):  
        try:        
            for step in np.arange(MAX_STEP):
                if coord.should_stop():
                        break
                val_acc_op, val_recall_op, val_precision_op = sess.run([valid_acc, val_recall, val_precision],feed_dict={pred:predicts})
                #val_logits,val_acc_op= sess.run([train_logits, valid_acc])
                #predicts=change_recall(val_logits)
                #print(np.shape(predicts))
                #val_recall_op, val_precision_op = sess.run([val_recall, val_precision],feed_dict={pred:predicts})
                #val_acc, cal_recall, val_precision = sess.run([valid__acc, val_recall, val_precision],feed_dict={x:valid_batch,y:valid_label_batch})
                #print(val_logits)
                print('Step %d, valid accuracy = %.2f%%, valid recall = %.2f%%, valid precision = %.2f%%' %(step, val_acc_op*100.0, val_recall_op*100.0, val_precision_op*100.0))
                
                        
#                 summary_str = sess.run(summary_op)
#                 valid_writer.add_summary(summary_str, step)

        except tf.errors.OutOfRangeError:
            print('Done validation -- epoch limit reached')

        finally:
            coord.request_stop()

There are 14355 TC
There are 434488 nonTC
INFO:tensorflow:Restoring parameters from ./train_resnet_lrnaug_balanced/model.ckpt-1564000
--------restore done---------
---------validation start----------
Step 0, valid accuracy = 82.52%, valid recall = 16.67%, valid precision = 2.17%
Step 1, valid accuracy = 84.57%, valid recall = 19.23%, valid precision = 2.72%
Step 2, valid accuracy = 83.30%, valid recall = 18.42%, valid precision = 3.80%
Step 3, valid accuracy = 85.16%, valid recall = 12.20%, valid precision = 2.72%
Step 4, valid accuracy = 85.35%, valid recall = 17.14%, valid precision = 3.26%
Step 5, valid accuracy = 82.32%, valid recall = 28.57%, valid precision = 3.26%
Step 6, valid accuracy = 82.62%, valid recall = 17.50%, valid precision = 3.80%
Step 7, valid accuracy = 82.91%, valid recall = 17.14%, valid precision = 3.26%
Step 8, valid accuracy = 86.72%, valid recall = 24.39%, valid precision = 5.43%
Step 9, valid accuracy = 85.35%, valid recall = 15.00%, valid precision = 3.26%


KeyboardInterrupt: 